# Document Retrival with Title Embedding and IDF on Texts (DR.TEIT)

In this method we used two scoring measure and aggregate them by a convex combination as below:
$$
λ*Similiarity_{Title Embedding} + (1-λ)*Similiarity_{TextIDF}
$$

We used LaBSE model for out embeddings. For computing title embedding similarities we used cosine similarity between query embeddings and each document's title embedding.

For the second part we used character-level (2gram to 8gram). We also trained our TF-IDF transformation matrix on the Multidoc2dial2022 documnets.

## Dataset
### Dataset Description

- **mutldoc2dial_doc.json** contains the documents that are indexed by key `domain` and `doc_id` . Each document instance includes the following,

  - `doc_id`: the ID of a document;
  - `title`: the title of the document;
  - `domain`: the domain of the document;
  - `doc_text`: the text content of the document (without HTML markups);
  - `doc_html_ts`: the document content with HTML markups and the annotated spans that are indicated by `text_id` attribute, which corresponds to `id_sp`.
  - `doc_html_raw`: the document content with HTML markups and without span annotations.
  - `spans`: key-value pairs of all spans in the document, with `id_sp` as key. Each span includes the following,
    - `id_sp`: the id of a  span as noted by `text_id` in  `doc_html_ts`;
    - `start_sp`/  `end_sp`: the start/end position of the text span in `doc_text`;
    - `text_sp`: the text content of the span.
    - `id_sec`: the id of the (sub)section (e.g. `<p>`) or title (`<h2>`) that contains the span.
    - `start_sec` / `end_sec`: the start/end position of the (sub)section in `doc_text`.
    - `text_sec`: the text of the (sub)section.
    - `title`: the title of the (sub)section.
    - `parent_titles`: the parent titles of the `title`.

- **multidoc2dial_dial_train.json** and **multidoc2dial_dial_validation.json**  contain the training and dev split of dialogue data that are indexed by key `domain` . Please note: **For test split, we only include a dummy file in this version.**

  Each dialogue instance includes the following,

  - `dial_id`: the ID of a dialogue;
  - `turns`: a list of dialogue turns. Each turn includes,
    - `turn_id`: the time order of the turn;
    - `role`: either "agent" or "user";READ
    - `da`: dialogue act;
    - `references`: a list of spans with `id_sp` ,  `label` and `doc_id`. `references` is empty if a turn is for indicating previous user query not answerable or irrelevant to the document. **Note** that labels "*precondition*"/"*solution*" are fuzzy annotations that indicate whether a span is for describing a conditional context or a solution.
    - `utterance`: the human-generated utterance based on the dialogue scene.
Downloading the training dataset:

In [1]:
def clean_text(text):
    """
    Clean the given text.

    :param text: input text
    :type text: str
    :return: cleaned string
    """
    return text.strip()

In [2]:
import json
with open('../../dataset/multidoc2dial/v1.0/multidoc2dial_doc.json', 'r') as f:
    multidoc2dial_doc = json.load(f)

### Extracting titles

In [3]:
titles = []
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        titles.append(doc_idx2)
titles

['Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0',
 'Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#2_0',
 'Benefits Planner: Disability | How You Apply | Social Security Administration#1_0',
 'Benefits Planner: Disability | How You Apply | Social Security Administration#2_0',
 'Learn what documents you will need to get a Social Security Card | Social Security Administration#2_0',
 'Learn what documents you will need to get a Social Security Card | Social Security Administration#3_0',
 'Learn what documents you will need to get a Social Security Card | Social Security Administration#4_0',
 'Learn what documents you will need to get a Social Security Card | Social Security Administration#5_0',
 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0',
 'Learn what documents you will need to get a Social Security Card | Social Security Administration#

In [4]:
len(titles)

488

### Extracting document texts

In [5]:
doc_texts_train = []
title_to_domain = {}
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        doc_texts_train.append(multidoc2dial_doc['doc_data'][doc_idx1]\
                                          [doc_idx2]['doc_text'].strip())
        title_to_domain[doc_idx2] = doc_idx1
doc_texts_train[10]

"Original Card for a Foreign Born U.S. Citizen Adult \nImportant You must present original documents or copies certified by the agency that issued them. We cannot accept photocopies or notarized copies. All documents must be current not expired. We cannot accept a receipt showing you applied for the document. \n\nWhat original documents do I need? \nCitizenship We can accept only certain documents as proof of U.S. citizenship. These include : U.S. passport ; Certificate of Naturalization N-550/N-570 ; Certificate of Citizenship N-560/N-561 ; Certificate of Report of Birth DS-1350 ; Consular Report of Birth Abroad FS-240, CRBA. Age You must present your foreign birth certificate if you have it or can get it within 10 days. If not , we will consider other documents such as your passport or a document issued by the Department of Homeland Security DHS as evidence of your age. Anyone age 12 or older requesting an original Social Security number must appear in person for an interview. We wil

In [9]:
len(doc_texts_train)

488

## Encoding the sentences
We use the LaBSE which is a Language-agnostic BERT Sentence Encoder (LaBSE) is a BERT-based model trained for sentence embedding for 109 languages. The pre-training process combines masked language modeling with translation language modeling. The model is useful for getting multilingual sentence embeddings and for bi-text retrieval.

In [6]:
!pip install --quiet transformers

In [38]:
from transformers import AutoTokenizer, AutoModel, AutoConfig
import numpy as np
import torch
from torch.nn.functional import normalize

from tqdm import tqdm

In [8]:
model_name = "setu4993/LaBSE"

In [9]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

Downloading:   0%|          | 0.00/300 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.98M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.18M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/576 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.75G [00:00<?, ?B/s]

### `get_embeddings`
In this method we extract the **pooler output** (Last layer hidden-state of the first token of the sequence (classification token) after further processing through the layers used for the auxiliary pretraining task. E.g. for BERT-family of models, this returns the classification token after processing through a linear layer and a tanh activation function. The linear layer weights are trained from the next sentence prediction (classification) objective during pretraining).

In [10]:
def get_embeddings(sentece):
    """
    Return embeddings based on encoder model

    :param sentence: input sentence(s)
    :type sentence: str or list of strs
    :return: embeddings
    """
    tokenized = tokenizer(sentece,
                                return_tensors="pt",
                                padding=True)
    with torch.no_grad():
        embeddings = model(**tokenized)
    
    return np.squeeze(np.array(embeddings.pooler_output))

### Title embedding

In [11]:
title_embeddings_file = 'doc_title_LaBSE_Embedding.npy'

if not os.path.exists(title_embeddings_file):
    title_embeddings = []
    for title in tqdm(titles):
        title_embeddings.append(get_embeddings(title))

    with open(title_embeddings_file, 'wb') as f:
        np.save(f, np.array(title_embeddings))
else:
    title_embeddings = np.load(title_embeddings_file)
    title_embeddings = list(title_embeddings)


In [12]:
import pickle
title_to_embeddings_file = 'title_to_embeddings.pkl'

if not os.path.exists(title_to_embeddings_file):
    title_to_embeddings = {}
    for title in tqdm(titles):
        title_to_embeddings[title] = get_embeddings(title)
    with open(title_to_embeddings_file, 'wb') as f:
        pickle.dump(title_to_embeddings, f)
else:
    with open(title_to_embeddings_file, 'rb') as f:
        title_to_embeddings = pickle.load(f)

## Calculating the IDF for each token

In [13]:
words_idf_file = 'IDFs.pkl'

if not os.path.exists(words_idf_file):
    # First getting all distinct words in all documents
    words = set()
    doc_texts_train_tokenized = []
    for doc in tqdm(doc_texts_train, desc="getting all words from documents"):
        tokenized_doc = [s.lower() for s in tokenizer.tokenize(doc)]
        doc_texts_train_tokenized.append(tokenized_doc) 
        words = set(tokenized_doc).union(words)

    # calculating each word IDF
    words2IDF = {}
    N_doc = len(doc_texts_train)
    for word in tqdm(words, desc="calculating words IDF scores"):
        n_word = 0
        for doc in doc_texts_train_tokenized:
            if word in doc:
                n_word += 1
        words2IDF[word] = np.log(N_doc / (n_word + 1))

    with open(words_idf_file, 'wb') as f:
        pickle.dump(words2IDF, f)

else:
    with open(words_idf_file, 'rb') as f:
        words2IDF = pickle.load(f)

In [14]:
len(words2IDF)

8446

In [15]:
def calc_idf_score(sentence):
    """
    Calculate the mean idf score for given sentence.
    (used to understand the contribution of the knowledge of each question
    questions with high frequent words are meaningless and we can ignore them
    roughly, which is done by this score.)

    :param sentence: input sentence
    :type sentence: str
    :return: mean idf score of sentence token
    """
    tokenzied_sentence = [s.lower() for s in tokenizer.tokenize(sentence)]
    score = 0
    for token in tokenzied_sentence:
        if token in words2IDF:
            score += words2IDF[token]
        else:
            score += np.log(N_doc)
    return score / len(tokenzied_sentence)

## Methods

### IDF only - Vanilla

This is the first method which we used for document retriver. In this method we just used similarity between query embeddign and document title embedding. We also used IDF scores as a factor for history queries by which we can pay more attention to the more informative query.

In [16]:
def predict_labelwise_doc_at_history(queries, k=1):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """
    similarities = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coef_sum = 0
    for query in queries:
        query_embd = get_embeddings(query)
        query_sim = list(map(lambda x: np.dot(x, query_embd) /
                            (np.linalg.norm(query_embd) * np.linalg.norm(x)),
                            title_embeddings))
        query_sim = np.array(query_sim)

        coef = calc_idf_score(query)
        coef_sum += coef
        similarities += coef * query_sim

    similarities = similarities / coef_sum
    best_k_idx = similarities.argsort()[::-1][:k]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    accuracy = similarities[best_k_idx]
    return accuracy, predictions

### IDF ordered

In the last method there wasn't any difference between query and histories - all sentences would be treat same regardless of their queried time. From now we will use a reweighting (multiplying query's score to $2^{-i}$ when $i$ is the index of query) system which favour last query more.

In [17]:
def predict_labelwise_doc_at_history_ordered(queries, k=1):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """
    similarities = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coef_sum = 0
    for i, query in enumerate(queries):
        query_embd = get_embeddings(query)
        query_sim = list(map(lambda x: np.dot(x, query_embd) /
                            (np.linalg.norm(query_embd) * np.linalg.norm(x)),
                            title_embeddings))
        query_sim = np.array(query_sim)

        coef = 2**(-i) * calc_idf_score(query)
        coef_sum += coef
        similarities += coef * query_sim

    similarities = similarities / coef_sum
    best_k_idx = similarities.argsort()[::-1][:k]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    accuracy = similarities[best_k_idx]
    return accuracy, predictions

### IDF ordered - softmaxed
In this method we changed a little bit. Instead of using coefitionts barely we apply the softmax function favouring the maximum score more. But results were not good. 

In [18]:
def predict_labelwise_doc_at_history_ordered_softmaxed(queries, k=1):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """
    similarities = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coefs = []
    sims = []
    for i, query in enumerate(queries):
        query_embd = get_embeddings(query)
        query_sim = list(map(lambda x: np.dot(x, query_embd) /
                            (np.linalg.norm(query_embd) * np.linalg.norm(x)),
                            title_embeddings))
        sims.append(np.array(query_sim))
        coefs.append(2**(-i) * calc_idf_score(query))
    
    # Softmax:
    coefs = np.array(list(map(lambda x: np.exp(-x), coefs)))
    coefs /= coefs.sum()
    coefs = list(coefs)

    for coef, sim in zip(coefs, sims):
        similarities += coef * sim
    best_k_idx = similarities.argsort()[::-1][:k]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    accuracy = similarities[best_k_idx]
    return accuracy, predictions

### IDF + self attention (cosine sim)

In this method we changed the reweighting method from power-order ($2^{-i}$) to a feed-forward self-attention mechanism. Here we just simply **use cosine similarity between each history and query as its coefficient**. As it's showing how far is it from the main query.

In [19]:
def predict_labelwise_doc_at_history_selfatt(queries, k=1):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """
    similarities = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coef_sum = 0
    query0_embd = get_embeddings(queries[0])
    for query in queries:
        query_embd = get_embeddings(query)
        query_sim = list(map(lambda x: np.dot(x, query_embd) /
                            (np.linalg.norm(query_embd) * np.linalg.norm(x)),
                            title_embeddings))
        query_sim = np.array(query_sim)

        coef = calc_idf_score(query) * np.dot(query0_embd, query_embd) / (np.linalg.norm(query_embd) * np.linalg.norm(query0_embd))
        coef_sum += coef
        similarities += coef * query_sim

    similarities = similarities / coef_sum
    best_k_idx = similarities.argsort()[::-1][:k]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    accuracy = similarities[best_k_idx]
    return accuracy, predictions

### DR. TEIT*

In this method we used two scoring measure and aggregate them by a convex combination as below:
$$
λ*Similiarity_{Title Embedding} + (1-λ)*Similiarity_{TextIDF}
$$

We used LaBSE model for our embeddings. For computing title embedding similarities we used cosine similarity between query embeddings and each document's title embedding.

For the second part we used character-level (2gram to 8gram). We also trained our TF-IDF transformation matrix on the Multidoc2dial2022 documnets.

**NOTE: In `predict_DR_TEIT` you may see a diffrent notation (`alpha`) but they are the same.**

#### TF-IDF Transformation Matrix Fitting

In [20]:
doc_texts_train = []
for doc_idx1 in multidoc2dial_doc['doc_data']:
    for doc_idx2 in multidoc2dial_doc['doc_data'][doc_idx1]:
        doc_texts_train.append(multidoc2dial_doc['doc_data'][doc_idx1]\
                                          [doc_idx2]['doc_text'].strip())
doc_texts_train[10]

"Original Card for a Foreign Born U.S. Citizen Adult \nImportant You must present original documents or copies certified by the agency that issued them. We cannot accept photocopies or notarized copies. All documents must be current not expired. We cannot accept a receipt showing you applied for the document. \n\nWhat original documents do I need? \nCitizenship We can accept only certain documents as proof of U.S. citizenship. These include : U.S. passport ; Certificate of Naturalization N-550/N-570 ; Certificate of Citizenship N-560/N-561 ; Certificate of Report of Birth DS-1350 ; Consular Report of Birth Abroad FS-240, CRBA. Age You must present your foreign birth certificate if you have it or can get it within 10 days. If not , we will consider other documents such as your passport or a document issued by the Department of Homeland Security DHS as evidence of your age. Anyone age 12 or older requesting an original Social Security number must appear in person for an interview. We wil

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidfVectorizer = TfidfVectorizer(strip_accents=None,
                                 analyzer='char',
                                 ngram_range=(2, 8),
                                 norm='l2',
                                 use_idf=True,
                                 smooth_idf=True)
tfidf_wm = tfidfVectorizer.fit_transform(doc_texts_train)

In [22]:
import pickle
with open('tfidfVectorizer.pkl', 'wb') as f:
    pickle.dump(tfidfVectorizer, f)

with open('tfidf_wm.pkl', 'wb') as f:
    pickle.dump(tfidf_wm, f)

In [29]:
len(tfidfVectorizer.get_feature_names_out())


1047632

#### DR. TEIT

the input is consisted of a list of queries, which is the current question and its history turns.
for each of the questions, we compute two similarity score for each of our documents, one of them is based on the pretrained LM and the other on is based on character level matching. Both of these scores will be weighted by a coefficient which is the `idf_score` of the query, defining how much meaning does the query contain. Then these scores will be summed up in a convex manner and the final matching score with all documents is computed. We return the result by sorting these scores.

In [23]:
def predict_DR_TEIT(queries, k=1, alpha=10):
    """
    Predict which document is matched to the given query.

    :param queries: input queries in time reversed order (latest first)
    :type queries: str (or list of strs)
    :param k: number of returning docs
    :type k: int 
    :return: return the document names and accuracies
    """

    idf_score = np.array(list(map(lambda x: 0.0, title_embeddings)))
    tfidf_score = np.array(list(map(lambda x: 0.0, title_embeddings)))
    coef_sum = 0
    for i, query in enumerate(queries):
        query_embd = get_embeddings(query)
        query_sim = list(map(lambda x: np.dot(x, query_embd) /
                            (np.linalg.norm(query_embd) * np.linalg.norm(x)),
                            title_embeddings))
        query_sim = np.array(query_sim)
        coef = 2**(-i) * calc_idf_score(query)
        coef_sum += coef

        idf_score += coef * query_sim
        tfidf_score += coef * np.squeeze(np.asarray(tfidf_wm @ tfidfVectorizer.transform([query]).todense().T))

    scores = (idf_score + alpha * tfidf_score) / coef_sum
    best_k_idx = scores.argsort()[::-1][:k]
    scores = scores[best_k_idx]
    predictions = list(map(lambda x: titles[x], best_k_idx))
    return (scores, predictions)

## Test
In the test dataset we just picked ones with **user** turn.

In [31]:
test_queries = ["I'm looking for information regarding benefits planning, can you help me?",
                "I want to know about the benefits plan for survivors, can you give me more information about this?",
                "What are Social Security credits?"]
test_labels = ["Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0",
               "Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0",
               "Benefits Planner: Survivors | Planning For Your Survivors | Social Security Administration#1_0"]

In [32]:
import json
with open('../../dataset/multidoc2dial/v1.0/multidoc2dial_dial_train.json', 'r') as f:
    multidoc2dial_dial_train = json.load(f)

In [33]:
doc_sentence_test = []
doc_label_test = []
for doc_idx1 in multidoc2dial_dial_train['dial_data']:
    for dial in multidoc2dial_dial_train['dial_data'][doc_idx1]:
        for turns in dial['turns']:
            if turns['role'] == "user":
                doc_sentence_test.append(turns['utterance'])
                doc_label_test.append(turns['references'][0]['doc_id'])

In [34]:
TEST_SIZE = len(doc_sentence_test)
TEST_SIZE

23399

In [35]:
TEST_SIZE = TEST_SIZE // 20   #   For making it faster

### IDF only - Vanilla

In [36]:
accs, preds = predict_labelwise_doc_at_history([test_queries[2],
                                               test_queries[1],
                                               test_queries[0]],
                                               k=5)
print(accs)
print(preds)
print('-' * 20)

[0.37842306 0.37675706 0.37660416 0.37555721 0.37375754]
['Benefits Planner: Survivors | How You Apply | Social Security Administration#1_0', 'Benefits Planner: Survivors | How You Apply | Social Security Administration#2_0', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2_3', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2', 'How To Apply For Social Security Disability Benefits#1_0']
--------------------


In [39]:
prec_at_500 = 0
prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in tqdm(range(2, TEST_SIZE), desc="running the test on document selection"):
    act_doc = doc_label_test[i]
    accs, preds = predict_labelwise_doc_at_history([doc_sentence_test[i],
                                                   doc_sentence_test[i-1],
                                                   doc_sentence_test[i-2]],
                                                   k=500)
    ranks.append(1 / (preds.index(act_doc) + 1))
    if act_doc == preds[0]:
        prec_at_1 += 1
    if act_doc in preds[:5]:
        prec_at_5 += 1
    if act_doc in preds[:10]:
        prec_at_10 += 1
    if act_doc in preds[:50]:
        prec_at_50 += 1
    if act_doc in preds[:100]:
        prec_at_100 += 1
    if act_doc in preds[:500]:
        prec_at_500 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
        print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
                     sample_till_now))

running the test on document selection:   0%|          | 0/1167 [00:00<?, ?it/s]


NameError: name 'N_doc' is not defined

### IDF ordered

In [53]:
accs, preds = predict_labelwise_doc_at_history_ordered([test_queries[2],
                                                        test_queries[1],
                                                        test_queries[0]],
                                                        k=5)
print(accs)
print(preds)
print('-' * 20)

[0.41813263 0.41414319 0.41318401 0.41088729 0.40784175]
['Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2_3', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#7_0_1_2', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2_3_4', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#12_0_1_2']
--------------------


In [55]:
prec_at_500 = 0
prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in tqdm(range(2, TEST_SIZE)):
    act_doc = doc_label_test[i]
    accs, preds = predict_labelwise_doc_at_history_ordered([doc_sentence_test[i],
                                                            doc_sentence_test[i-1],
                                                            doc_sentence_test[i-2]],
                                                            k=500)
    ranks.append(1 / (preds.index(act_doc) + 1))
    if act_doc == preds[0]:
        prec_at_1 += 1
    if act_doc in preds[:5]:
        prec_at_5 += 1
    if act_doc in preds[:10]:
        prec_at_10 += 1
    if act_doc in preds[:50]:
        prec_at_50 += 1
    if act_doc in preds[:100]:
        prec_at_100 += 1
    if act_doc in preds[:500]:
        prec_at_500 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
        print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
                     sample_till_now))

  9%|▊         | 100/1167 [00:36<07:31,  2.36it/s]

MRR: mean=0.144511049798484, var=0.08118642286253466
Prec@(1) = 0.09 | Prec@(5) = 0.16 | Prec@(10) = 0.28 | Prec@(50) = 0.53 | Prec@(100) = 0.72 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 100


 17%|█▋        | 200/1167 [01:05<05:04,  3.18it/s]

MRR: mean=0.15031672380406347, var=0.08741299578833683
Prec@(1) = 0.1 | Prec@(5) = 0.165 | Prec@(10) = 0.275 | Prec@(50) = 0.545 | Prec@(100) = 0.76 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 200


 26%|██▌       | 300/1167 [01:37<03:45,  3.85it/s]

...

 86%|████████▌ | 1000/1167 [05:01<00:47,  3.52it/s]

MRR: mean=0.2309306323833912, var=0.11833709052365163
Prec@(1) = 0.146 | Prec@(5) = 0.307 | Prec@(10) = 0.406 | Prec@(50) = 0.645 | Prec@(100) = 0.822 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 1000


 94%|█████████▍| 1100/1167 [05:28<00:17,  3.81it/s]

MRR: mean=0.23472752180863185, var=0.12019851748856636
Prec@(1) = 0.15 | Prec@(5) = 0.31 | Prec@(10) = 0.4109090909090909 | Prec@(50) = 0.6554545454545454 | Prec@(100) = 0.8318181818181818 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 1100


100%|██████████| 1167/1167 [05:48<00:00,  3.35it/s]


### IDF ordered - softmaxed

In [56]:
accs, preds = predict_labelwise_doc_at_history_ordered_softmaxed([test_queries[2],
                                                        test_queries[1],
                                                        test_queries[0]],
                                                        k=5)
print(accs)
print(preds)
print('-' * 20)

[0.37505371 0.37495329 0.36257431 0.36173663 0.36112786]
['Benefits Planner: Survivors | How You Apply | Social Security Administration#1_0', 'Benefits Planner: Survivors | How You Apply | Social Security Administration#2_0', 'Benefits Planner: Disability | How You Apply | Social Security Administration#2_0', 'Learn About Retirement Benefits | SSA#1_0', 'Benefits Planner: Disability | How You Apply | Social Security Administration#1_0']
--------------------


In [57]:
prec_at_500 = 0
prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in tqdm(range(2, TEST_SIZE)):
    act_doc = doc_label_test[i]
    accs, preds = predict_labelwise_doc_at_history_ordered_softmaxed([doc_sentence_test[i],
                                                            doc_sentence_test[i-1],
                                                            doc_sentence_test[i-2]],
                                                            k=500)
    ranks.append(1 / (preds.index(act_doc) + 1))
    if act_doc == preds[0]:
        prec_at_1 += 1
    if act_doc in preds[:5]:
        prec_at_5 += 1
    if act_doc in preds[:10]:
        prec_at_10 += 1
    if act_doc in preds[:50]:
        prec_at_50 += 1
    if act_doc in preds[:100]:
        prec_at_100 += 1
    if act_doc in preds[:500]:
        prec_at_500 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
        print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
                     sample_till_now))

  9%|▊         | 100/1167 [00:28<04:38,  3.84it/s]

MRR: mean=0.09269418454220091, var=0.041466296907041615
Prec@(1) = 0.04 | Prec@(5) = 0.12 | Prec@(10) = 0.24 | Prec@(50) = 0.44 | Prec@(100) = 0.6 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 100


 17%|█▋        | 200/1167 [00:57<05:03,  3.19it/s]

MRR: mean=0.090842020703304, var=0.042118003791540175
Prec@(1) = 0.04 | Prec@(5) = 0.11 | Prec@(10) = 0.21 | Prec@(50) = 0.44 | Prec@(100) = 0.62 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 200


 26%|██▌       | 300/1167 [01:27<05:49,  2.48it/s]

...

 86%|████████▌ | 1000/1167 [04:54<00:46,  3.60it/s]

MRR: mean=0.18189723053882215, var=0.09835962345936167
Prec@(1) = 0.113 | Prec@(5) = 0.238 | Prec@(10) = 0.321 | Prec@(50) = 0.576 | Prec@(100) = 0.778 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 1000


 94%|█████████▍| 1100/1167 [05:21<00:17,  3.73it/s]

MRR: mean=0.1754127208713634, var=0.09439787862058427
Prec@(1) = 0.10727272727272727 | Prec@(5) = 0.23 | Prec@(10) = 0.31545454545454543 | Prec@(50) = 0.5763636363636364 | Prec@(100) = 0.78 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 1100


100%|██████████| 1167/1167 [05:39<00:00,  3.44it/s]


### IDF + self attention (cosine sim)

In [58]:
accs, preds = predict_labelwise_doc_at_history_selfatt([test_queries[2],
                                                        test_queries[1],
                                                        test_queries[0]],
                                                        k=5)
print(accs)
print(preds)
print('-' * 20)

[0.42571371 0.42203484 0.42132396 0.41824856 0.4159181 ]
['Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2_3', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#7_0_1_2', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#6_0_1_2_3_4', 'Learn what documents you will need to get a Social Security Card | Social Security Administration#12_0_1_2']
--------------------


In [59]:
prec_at_500 = 0
prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in tqdm(range(2, TEST_SIZE)):
    act_doc = doc_label_test[i]
    accs, preds = predict_labelwise_doc_at_history_selfatt([doc_sentence_test[i],
                                                            doc_sentence_test[i-1],
                                                            doc_sentence_test[i-2]],
                                                            k=500)
    ranks.append(1 / (preds.index(act_doc) + 1))
    if act_doc == preds[0]:
        prec_at_1 += 1
    if act_doc in preds[:5]:
        prec_at_5 += 1
    if act_doc in preds[:10]:
        prec_at_10 += 1
    if act_doc in preds[:50]:
        prec_at_50 += 1
    if act_doc in preds[:100]:
        prec_at_100 += 1
    if act_doc in preds[:500]:
        prec_at_500 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
        print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
                     sample_till_now))

  9%|▊         | 100/1167 [00:39<05:58,  2.98it/s]

MRR: mean=0.13712179858232124, var=0.06804061197193524
Prec@(1) = 0.07 | Prec@(5) = 0.17 | Prec@(10) = 0.31 | Prec@(50) = 0.49 | Prec@(100) = 0.72 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 100


 17%|█▋        | 200/1167 [01:17<06:37,  2.43it/s]

MRR: mean=0.1407645002818882, var=0.07746681079191914
Prec@(1) = 0.085 | Prec@(5) = 0.165 | Prec@(10) = 0.28 | Prec@(50) = 0.505 | Prec@(100) = 0.755 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 200


 26%|██▌       | 300/1167 [01:54<04:46,  3.02it/s]

...

 86%|████████▌ | 1000/1167 [06:23<01:02,  2.68it/s]

MRR: mean=0.21360611068958624, var=0.11018580068420077
Prec@(1) = 0.133 | Prec@(5) = 0.289 | Prec@(10) = 0.384 | Prec@(50) = 0.623 | Prec@(100) = 0.816 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 1000


 94%|█████████▍| 1100/1167 [07:01<00:24,  2.70it/s]

MRR: mean=0.21816521586092671, var=0.11363966031532553
Prec@(1) = 0.1390909090909091 | Prec@(5) = 0.2909090909090909 | Prec@(10) = 0.3845454545454545 | Prec@(50) = 0.6281818181818182 | Prec@(100) = 0.8227272727272728 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 1100


100%|██████████| 1167/1167 [07:25<00:00,  2.62it/s]


### DR.TEIT

In [60]:
prec_at_500 = 0
prec_at_100 = 0
prec_at_50 = 0
prec_at_10 = 0
prec_at_5 = 0
prec_at_1 = 0
sample_till_now = 0
ranks = []
for i in tqdm(range(2, TEST_SIZE)):
    act_doc = doc_label_test[i]
    accs, preds = predict_DR_TEIT([doc_sentence_test[i],
                                   doc_sentence_test[i-1],
                                   doc_sentence_test[i-2]],
                                   k=500,
                                   alpha=10)
    ranks.append(1 / (preds.index(act_doc) + 1))
    if act_doc == preds[0]:
        prec_at_1 += 1
    if act_doc in preds[:5]:
        prec_at_5 += 1
    if act_doc in preds[:10]:
        prec_at_10 += 1
    if act_doc in preds[:50]:
        prec_at_50 += 1
    if act_doc in preds[:100]:
        prec_at_100 += 1
    if act_doc in preds[:500]:
        prec_at_500 += 1
    sample_till_now += 1
    if sample_till_now % 100 == 0:
        print("MRR: mean={}, var={}".format(np.array(ranks).mean(), np.array(ranks).var()))
        print("Prec@(1) = {} | Prec@(5) = {} | Prec@(10) = {} | Prec@(50) = {} | Prec@(100) = {} | Prec@(500) = {} | NUMBER_OF_SAMPLES = {}".\
              format(prec_at_1 / sample_till_now, prec_at_5 / sample_till_now,
                     prec_at_10 / sample_till_now, prec_at_50 / sample_till_now,
                     prec_at_100 / sample_till_now, prec_at_500 / sample_till_now,
                     sample_till_now))

  9%|▊         | 100/1167 [00:40<06:19,  2.81it/s]

MRR: mean=0.6718437625222385, var=0.14197122368318632
Prec@(1) = 0.54 | Prec@(5) = 0.81 | Prec@(10) = 0.9 | Prec@(50) = 0.99 | Prec@(100) = 0.99 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 100


 17%|█▋        | 200/1167 [01:19<06:28,  2.49it/s]

MRR: mean=0.7621047653992961, var=0.12164531618233917
Prec@(1) = 0.66 | Prec@(5) = 0.875 | Prec@(10) = 0.925 | Prec@(50) = 0.98 | Prec@(100) = 0.99 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 200


 26%|██▌       | 300/1167 [01:59<05:00,  2.88it/s]

...

 86%|████████▌ | 1000/1167 [06:34<01:04,  2.60it/s]

MRR: mean=0.7245883303501048, var=0.13281883289201785
Prec@(1) = 0.612 | Prec@(5) = 0.864 | Prec@(10) = 0.918 | Prec@(50) = 0.967 | Prec@(100) = 0.982 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 1000


 94%|█████████▍| 1100/1167 [07:10<00:24,  2.72it/s]

MRR: mean=0.7253423659263777, var=0.1344064622774479
Prec@(1) = 0.6163636363636363 | Prec@(5) = 0.86 | Prec@(10) = 0.9136363636363637 | Prec@(50) = 0.9636363636363636 | Prec@(100) = 0.980909090909091 | Prec@(500) = 1.0 | NUMBER_OF_SAMPLES = 1100


100%|██████████| 1167/1167 [07:34<00:00,  2.56it/s]


## Results

At last we have resutls as follows:


| Method | @1 | @5 | @10 | @50 | @100 | MRR (mean, var) |
|:------:|:------:|:------:|:-------:|:-------:|:--------:|:---:|
| IDF - vanilla | 13% | 30% | 39% | 64% | 83% | (0.22, 0.11) |
| IDF - power-order | 15% | 31% | 41% | 65% | 83% | (0.23, 0.12) |
| IDF - power-order (softmax) | 10.7% | 23% | 31% | 57.6% | 78% | (0.18, 0.09) |
| IDF - self-attention | 13.9% | 29% | 38% | 62% | 82% | (0.22, 0.11) |
| **DR. TEIT** | **61.6%** | **86%** | **91%** | **96%** | **98%** | **(0.72, 0.13)** |

It shows that title informations were not enough for document retrieval.

# drafts

In [58]:
tfidf_wm.shape

(488, 1047632)

In [59]:
answers = tfidfVectorizer.transform(["Original Card for a Foreign Born U.S. Citizen Adult",
                                     "Hello world from far beyound!"]).todense()
query = tfidfVectorizer.transform(["Hello!"]).todense()

In [60]:
print(answers.shape, query.shape)

(2, 1047632) (1, 1047632)


In [61]:
import numpy as np
answers_sim = np.squeeze(np.asarray(tfidf_wm @ answers.T))
query_sim = np.squeeze(np.asarray(tfidf_wm @ query.T))

In [62]:
print(answers_sim.shape, query_sim.shape)

(488, 2) (488,)


In [63]:
list(map(lambda x: np.dot(x, query_sim) /
        (np.linalg.norm(query_sim) * np.linalg.norm(x)),
        answers_sim.T))

[0.7506911990367025, 0.934114716518692]

In [1]:
from transformers import BertTokenizer, BertModel
import torch

model_name = ["t5-base", "bert-base-uncased"][0]

tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
print("inputs", inputs)

pooler = outputs.pooler_output
print("last_hidden_states", last_hidden_states.shape)

print("pooler",pooler.shape)
with torch.no_grad():
    print(np.squeeze(np.array(pooler)).shape)

OSError: Can't load tokenizer for 't5-base'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 't5-base' is the correct path to a directory containing all relevant files for a BertTokenizer tokenizer.